In [ ]:
import pandas as pd

import csv
import time
import datetime

from tqdm import tqdm
from datetime import datetime as dt
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

In [ ]:
def getPlayerURL(playerName, role):
    namesMapping = {
        'Ghost': 'Ghost (Jang Yong-jun)',
        'Rich': 'Rich (Lee Jae-won)',
        'knight': 'Knight (Zhuo Ding)',
        'Bin': 'Bin (Chen Ze-Bin)',
        'Angel': 'Angel (Xiang Tao)',
        'Aki': 'Aki (Mao An)',
        'Wuming': 'Wuming (Wang Xin)',
        'Eric': 'Eric (He Qiang)',
        'Viper': 'Viper (Park Do-hyeon)',
        'Tuesday': 'Tuesday (Jean-Sébastien Thery)',
        'Apollo': 'Apollo (Apollo Price)',
        'John': 'John (John Holtzclaw)',
        'King': {
            'jng': 'King (Luka Kralj)',
            'sup': 'King (Trevor Roy)'
        },
        'Matty': 'Matty (Mathieu Breton)',
        'Ziv': 'Ziv (Chen Yi)',
        'QiuQiu': 'QiuQiu (Zhang Ming)',
        'Zeka': 'Zeka (Kim Geon-woo)',
        'Mark': 'Mark (Ling Xu)',
        'bless': 'Bless (Xiang Yi-Tong)',
        'Captain': 'Captain (Luo Fei-Yang)',
        'Wei': 'Wei (Yan Yang-Wei)',
        'Ming': 'Ming (Shi Sen-Ming)',
        'Phoenix': 'Phoenix (Maarten Van Dyck)',
        'Aesthetic': 'Aesthetic (Frank Norqvist)',
        'Night': 'Night (Jeroen Segers)',
        'Miś': 'Miś (Michał Kopacz)',
        'Young': 'Young (Young Choi)',
        'Albus NoX': 'Albus NoX (Stepan Titov)',
        'Circus': 'Circus (Jacob Yoh)',
        'Diesel': 'Diesel (Darrell Jenkins)',
        'Aspect': 'Aspect (Joshua Lee)',
        'Shiro': 'Shiro (Kyle Sakamaki)',
        'Riku': 'Riku (Henry Nguyen)',
        'Eclipse': 'Eclipse (Yujie Wu)',
        'Gweiss': 'Gweiss (Garrett Weiss)',
        'Scholar': 'Scholar (Preston Hall)',
        'APA': 'APA (Eain Stearns)',
        'Winter': 'Winter (Olivier Lapointe)',
        'Hunter': 'Hunter (Hunter Meyer)',
        'Hyper': 'Hyper (Young Seo)',
        'Barrage': 'Barrage (Luc van Gestel)',
        'NooB': 'NooB (Léo Pitrey)',
        'GGsir': 'GG Sir',
        'Seal': 'Seal (Fabian de Lint)',
        'Shadow': 'Shadow (Li Hao)',
        'Doran': 'Doran (Choi Hyeon-joon)',
        'HyBriD': 'HyBriD (Lee Woo-jin)',
        'Ahn': 'Ahn (An Shan-Ye)',
        'CJ': 'CJ (Yang Chong-Jiang)',
        'Arthur': 'Arthur (Park Mi-reu)',
        'Jerry': 'Jerry (Zhou Ke-Xue)',
        'Sleepy': 'Sleepy (Wang He-Yong)',
        'Tarzan': 'Tarzan (Lee Seung-yong)',
        'Light': 'Light (Wang Guang-Yu)',
        'Taxer': 'Taxer (Christian Vendelbo)',
        'Click': 'Click (Vittorio Massolo)',
        'aki': 'Aki (Daniel Lauri)',
        'Lotus': 'Lotus (Dušan Nedeljković)',
        'Jay': 'Jay (Chen Bo)',
        'FATE': 'FATE (Yoo Su-hyeok)',
        'River': 'River (Luo Bai-Jiang)',
        'QingZhi': 'QingZhi (Lin Han)',
        'BAO': 'BAO (Jeong Hyeon-woo)',
        'Becca': 'Becca (Son Min-woo)',
        'Dread': 'Dread (Lee Jin-hyeok)',
        'Fly': 'Fly (Song Yong-jun)',
        'Kane': 'Kane (Chen Hao)',
        'Sora': 'Sora (Liu Zhi-Long)',
        'demon': 'Demon (Wu Yue-Wei)',
        'Cube': 'Cube (Dai Yi)',
        'Hachi': 'Hachi (Davy de Graaf)',
        'Closer': 'Closer (Can Çelik)',
        'Solo': 'Solo (Colin Earnest)',
        'Griffin': 'Griffin (Raymond Griffin)',
        'Vulcan': 'Vulcan (Philippe Laflamme)',
        'Neo': 'Neo (Toàn Trần)',
        'Diamond': 'Diamond (David Bérubé)',
        'Ye': 'Ye (Ji Xin-Yu)',
        'Naruto': 'Naruto (Nie Hao)',
        'Trigger': 'Trigger (Kim Eui-joo)',
        'Chance': 'Chance (Pei Piao)',
        'Trigo': 'Trigo (Matheus Trigo)',
        'Bankai': 'Bankai (Renan Pirone)',
        'Aegis': 'Aegis (Gabriel Lemos)',
        'Avenger': 'Avenger (Adriano Perassoli)',
        'Jojo': 'Jojo (Gabriel Dzelme)',
        'Envy': 'Envy (Bruno Farias)',
        'Hawk': 'Hawk (Gabriel Gomes)',
        'Spark': 'Spark (Lucas Keith)',
        'Eragon': 'Eragon (Adam Harney)',
        'Lonely': 'Lonely (Han Gyu-joon)',
        'Dice': 'Dice (Hong Do-hyeon)',
        'Kael': 'Kael (Kim Jin-hong)',
        'Clear': 'Clear (Song Hyeon-min)',
        'Jun': 'Jun (Yoon Se-joon)',
        'Noah': 'Noah (Oh Hyeon-taek)',
        'Thanatos': 'Thanatos (Park Seung-gyu)',
        'Storm': 'Storm (Lee Jae-dong)',
        'Valkyrie': 'Valkyrie (Ko Joon-yeong)',
        'Dream': 'Dream (Tan Wen-Xiang)',
        'FIESTA': 'FIESTA (An Hyeon-seo)',
        'LOCKIE': 'Tsigas',
        'Deluxe': 'Deluxe (Klemen Papež)',
        'Hades': 'Hades (Maik Jonker)',
        'Nyx': 'Nyx (Óscar Ruiz Vargas)',
        'Marky': 'Marky (Pedro José Serrano)',
        'Sharp': 'Sharp (Anders Lilleengen)',
        'Sven': 'Sven (Sven Olejnikow)',
        'Eren': 'Eren (Nguyễn Đức Anh)',
        'Scarface': 'Scarface (Daniel Aitbelkacem)',
        'Lion': 'Lion (Leon Anton)',
        'SMILEY': 'SMILEY (Ludvig Granquist)',
        'Adam': 'Adam (Adam Maanane)',
        'Rabble': 'Rabble (Jochem van Graafeiland)',
        'Rogue': 'Rogue (Jake Sharwood)',
        'Scarlet': 'Scarlet (Marcel Wiederhofer)',
        'Tyrin': 'Tyrin (William Portugal)',
        'Royal': 'Royal (Alexandru Mihai Pricu)',
        'Dioge': 'Dioge (Diogenes Barbosa)',
        'Flare': 'Flare (Luiz Felipe Lobo)',
        'Prime': 'Prime (Olivier Payet)',
        'asta': 'Asta (Wyllian Adriano)',
        'Bounty': 'Bounty (Gabriel Donner)',
        'Raven': 'Ravenzin',
        'Leo': 'Leo (Han Gyeo-re)',
        'Air': 'Air (Shenghao He)',
        'Yeon': 'Yeon (Sean Sung)',
        'Spawn': 'Spawn (Trevor Kerr-Taylor)',
        'Copy': 'Copy (Jouhan Pathmanathan)'
    }

    if playerName in namesMapping:
        if isinstance(namesMapping[playerName], str):
            return namesMapping[playerName]
        else:
            return namesMapping[playerName][role]

    return playerName

In [ ]:
def crawlerPlayerInfos(playerName, playingChampion, endDate, game, role):    
    gp = 'FIRST TIME PLAYING'
    wr ='FIRST TIME PLAYING'
    kda = 'FIRST TIME PLAYING'
    firstTimePlaying = False
    playerName = getPlayerURL(playerName, role)

    chrome_path = r'dependency/chromedriver'
    options = webdriver.ChromeOptions()
    options.headless = True
    options.add_argument('window-size=1400,600')
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    driver = webdriver.Chrome(executable_path=chrome_path, options=options)
    driver.get("https://oracleselixir.com/player/{}/championPool".format(playerName))

    # Aguardando carregamento da página e exibição do input de Start Date
    wait = WebDriverWait(driver, 5)
    try:
        start_date_input = wait.until(ec.presence_of_element_located(
                (By.XPATH, '//*[@id="start_date"]')), message='Campo "Start Date" era esperado na busca do jogador {} e não foi encontrado.'.format(playerName))
        start_date_input.send_keys('01/01/2021')
    except TimeoutException as ex:
        with open('outputs/incorrect-names.txt', 'a', encoding='utf-8') as f:
            f.write("{} - {}\n".format(playerName, game))
        return []

    # Aguardando carregamento da página e exibição do input de End Date
    start_date_input = wait.until(ec.presence_of_element_located(
            (By.XPATH, '//*[@id="end_date"]')), message='Campo "Start Date" era esperado na busca do jogador {} e não foi encontrado.'.format(playerName))
    start_date_input.send_keys(endDate)

    # Aguardando carregamento da página e exibição dos resultados
    time.sleep(5)
    
    if ( len(driver.find_elements_by_xpath("//*[text()=\"{}\"]/following::div/following::div".format(playingChampion))) != 0):
        # Obtendo valores de GP, W% e KDA
        firstTimePlaying = True
        games_played_element = driver.find_elements_by_xpath("//*[text()=\"{}\"]/following::div/following::div".format(playingChampion))[0]
        win_rate_element = driver.find_elements_by_xpath("//*[text()=\"{}\"]/following::div/following::div/following::div".format(playingChampion))[0]
        kda_element = driver.find_elements_by_xpath("//*[text()=\"{}\"]/following::div/following::div/following::div/following::div/following::div".format(playingChampion))[0]

        gp = games_played_element.text
        wr = win_rate_element.text
        kda = kda_element.text
  

    driver.quit()

    return gp, wr, kda, firstTimePlaying

In [ ]:
df = pd.read_csv("data/mess_dataset_20210515.csv")
games = df.gameid.drop_duplicates()

header = 'game,bluetop,bluetopgp,bluetopwr,bluetopkda,bluejungle,bluejunglegp,bluejunglewr,bluejunglekda,bluemid,bluemidgp,bluemidwr,bluemidkda,blueadc,blueadcgp,blueadcwr,blueadckda,bluesupport,bluesupportgp,bluesupportwr,bluesupportkda,redtop,redtopgp,redtopwr,redtopkda,redjungle,redjunglegp,redjunglewr,redjunglekda,redmid,redmidgp,redmidwr,redmidkda,redadc,redadcgp,redadcwr,redadckda,redsupport,redsupportgp,redsupportwr,redsupportkda,result\n'
with open('outputs/dataset_20210515.csv', mode='a') as dataset2021:
    dataset2021.write(header)
    
def processGames(game):
    
    if isinstance(game, str):

        # GAME DATETIME
        endDate = dt.strptime(df[(df['gameid']==game)].date.values[0], '%Y-%m-%d %H:%M:%S')
        formatedEndDate = "{:02d}/{:02d}/{}".format(endDate.day - 1,endDate.month,endDate.year)

        # BLUE TEAM
        blueTop = df[(df['gameid']==game) & (df['side']=='Blue') & (df['position']=='top')].champion.values[0]
        blueTopPlayer = df[(df['gameid']==game) & (df['side']=='Blue') & (df['position']=='top')].player.values[0]
        if (blueTopPlayer != 'unknown player'):
            crawler = crawlerPlayerInfos(blueTopPlayer, blueTop, formatedEndDate, game, 'top')
            if (crawler):
              blueTopGP = crawler[0]
              blueTopWR = crawler[1]
              blueTopKDA = crawler[2]
              firstTimePlaying = crawler[3]
            else:
              return
        else:
            return

        blueJungle = df[(df['gameid']==game) & (df['side']=='Blue') & (df['position']=='jng')].champion.values[0]
        blueJunglePlayer = df[(df['gameid']==game) & (df['side']=='Blue') & (df['position']=='jng')].player.values[0]
        if (blueJunglePlayer != 'unknown player'):
            crawler = crawlerPlayerInfos(blueJunglePlayer, blueJungle, formatedEndDate, game, 'jng')
            if (crawler):
              blueJungleGP = crawler[0]
              blueJungleWR = crawler[1]
              blueJungleKDA = crawler[2]
              firstTimePlaying = crawler[3]
            else:
              return
        else:
            return

        blueMid = df[(df['gameid']==game) & (df['side']=='Blue') & (df['position']=='mid')].champion.values[0]
        blueMidPlayer = df[(df['gameid']==game) & (df['side']=='Blue') & (df['position']=='mid')].player.values[0]
        if (blueMidPlayer != 'unknown player'):
            crawler = crawlerPlayerInfos(blueMidPlayer, blueMid, formatedEndDate, game, 'mid')
            if (crawler):
              blueMidGP = crawler[0]
              blueMidWR = crawler[1]
              blueMidKDA = crawler[2]
              firstTimePlaying = crawler[3]
            else:
              return
        else:
            return

        blueCarry = df[(df['gameid']==game) & (df['side']=='Blue') & (df['position']=='bot')].champion.values[0]
        blueCarryPlayer = df[(df['gameid']==game) & (df['side']=='Blue') & (df['position']=='bot')].player.values[0]
        if (blueCarryPlayer != 'unknown player'):
            crawler = crawlerPlayerInfos(blueCarryPlayer, blueCarry, formatedEndDate, game, 'bot')
            if (crawler):
              blueCarryGP = crawler[0]
              blueCarryWR = crawler[1]
              blueCarryKDA = crawler[2]
              firstTimePlaying = crawler[3]
            else:
              return
        else:
            return

        blueSupp = df[(df['gameid']==game) & (df['side']=='Blue') & (df['position']=='sup')].champion.values[0]
        blueSuppPlayer = df[(df['gameid']==game) & (df['side']=='Blue') & (df['position']=='sup')].player.values[0]
        if (blueSuppPlayer != 'unknown player'):
            crawler = crawlerPlayerInfos(blueSuppPlayer, blueSupp, formatedEndDate, game, 'sup')
            if (crawler):
              blueSuppGP = crawler[0]
              blueSuppWR = crawler[1]
              blueSuppKDA = crawler[2]
              firstTimePlaying = crawler[3]
            else:
              return
        else:
            return

        # RED TEAM
        redTop = df[(df['gameid']==game) & (df['side']=='Red') & (df['position']=='top')].champion.values[0]
        redTopPlayer = df[(df['gameid']==game) & (df['side']=='Red') & (df['position']=='top')].player.values[0]
        if (redTopPlayer != 'unknown player'):
            crawler = crawlerPlayerInfos(redTopPlayer, redTop, formatedEndDate, game, 'top')
            if (crawler):
              redTopGP = crawler[0]
              redTopWR = crawler[1]
              redTopKDA = crawler[2]
              firstTimePlaying = crawler[3]
            else:
              return
        else:
            return

        redJungle = df[(df['gameid']==game) & (df['side']=='Red') & (df['position']=='jng')].champion.values[0]
        redJunglePlayer = df[(df['gameid']==game) & (df['side']=='Red') & (df['position']=='jng')].player.values[0]
        if (redJunglePlayer != 'unknown player'):
            crawler = crawlerPlayerInfos(redJunglePlayer, redJungle, formatedEndDate, game, 'jng')
            if (crawler):
              redJungleGP = crawler[0]
              redJungleWR = crawler[1]
              redJungleKDA = crawler[2]
              firstTimePlaying = crawler[3]
            else:
              return
        else:
            return

        redMid = df[(df['gameid']==game) & (df['side']=='Red') & (df['position']=='mid')].champion.values[0]
        redMidPlayer = df[(df['gameid']==game) & (df['side']=='Red') & (df['position']=='mid')].player.values[0]
        if (redMidPlayer != 'unknown player'):
            crawler = crawlerPlayerInfos(redMidPlayer, redMid, formatedEndDate, game, 'mid')
            if (crawler):
              redMidGP = crawler[0]
              redMidWR = crawler[1]
              redMidKDA = crawler[2]
              firstTimePlaying = crawler[3]
            else:
              return
        else:
            return

        redCarry = df[(df['gameid']==game) & (df['side']=='Red') & (df['position']=='bot')].champion.values[0]
        redCarryPlayer = df[(df['gameid']==game) & (df['side']=='Red') & (df['position']=='bot')].player.values[0]
        if (redCarryPlayer != 'unknown player'):
            crawler = crawlerPlayerInfos(redCarryPlayer, redCarry, formatedEndDate, game, 'bot')
            if (crawler):
              redCarryGP = crawler[0]
              redCarryWR = crawler[1]
              redCarryKDA = crawler[2]
              firstTimePlaying = crawler[3]
            else:
              return
        else:
            return
        
        redSupp = df[(df['gameid']==game) & (df['side']=='Red') & (df['position']=='sup')].champion.values[0]
        redSuppPlayer = df[(df['gameid']==game) & (df['side']=='Red') & (df['position']=='sup')].player.values[0]
        if (redSuppPlayer != 'unknown player'):
            crawler = crawlerPlayerInfos(redSuppPlayer, redSupp, formatedEndDate, game, 'sup')
            if (crawler):
              redSuppGP = crawler[0]
              redSuppWR = crawler[1]
              redSuppKDA = crawler[2]
              firstTimePlaying = crawler[3]
            else:
              return
        else:
            return

        # RESULT
        result = df[(df['gameid']==game) & (df['side']=='Red') & (df['position']=='sup')].result.values[0]

        # WRITING TO DATASET FILE        
        with open('outputs/dataset_20210515.csv', mode='a', newline="") as dataset2021:
            datasetWriter = csv.writer(dataset2021, delimiter=',')
            datasetWriter.writerow([game,blueTop, blueTopGP, blueTopWR, blueTopKDA, blueJungle, blueJungleGP, blueJungleWR, blueJungleKDA, blueMid, blueMidGP, blueMidWR, blueMidKDA, blueCarry, blueCarryGP, blueCarryWR, blueCarryKDA, blueSupp, blueSuppGP, blueSuppWR, blueSuppKDA, redTop,redTopGP, redTopWR, redTopKDA, redJungle, redJungleGP, redJungleWR, redJungleKDA, redMid, redMidGP, redMidWR, redMidKDA, redCarry, redCarryGP, redCarryWR, redCarryKDA, redSupp, redSuppGP, redSuppWR, redSuppKDA, result])        


In [ ]:
for game in tqdm(games):
    processGames(game)